# 超参数

In [4]:
import numpy as np
from sklearn import datasets

digits = datasets.load_digits()
X = digits.data
y = digits.target

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [7]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(X_train, y_train)
knn_clf.score(X_test, y_test)

c:\program files\python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


0.9888888888888889

## kNN的超参数:
调参    
如果最好的参数在我们寻找的边界上, 就要扩大搜索的边界    
- k 离最近的k的点数
- method: 要不要给距离加上权重 (距离的倒数, 离得越近权重越大)(同时能解决平票的问题)
- p (超参数之间可能有依赖关系, 例如有p的话, 就得考虑距离, 即要设置method=distance)

#### 补充: 马科夫斯基距离
Minkowsk metric

$\left ( \sum\limits_{i=1}^{n}\left | X_{i}^{(a)}-X_{i}^{(b)} \right |^{p} \right )^{\frac{1}{p}}$    
当p == 1时 为曼哈顿距离   
当p == 2时 为欧拉距离

### 找最好的k和method

In [23]:
best_method = "" # 要不要权重
best_score = 0.0
best_k = -1
for method in ["uniform", "distance"]:
    for k in range(1, 11):
        knn_clf = KNeighborsClassifier(n_neighbors=k, weights=method)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test) 
        if score > best_score:
            best_method = method
            best_score = score
            best_k = k
print(best_method)
print(best_score)
print(best_k)

uniform
0.9916666666666667
4


### 探索马科夫斯基距离相应的p

In [25]:
%%time
best_p = -1
best_score = 0.0
best_k = -1
for k in range(1, 11):
    for p in range(1, 6):
        knn_clf = KNeighborsClassifier(n_neighbors=k, weights="distance", p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test) 
        if score > best_score:
            best_p = p
            best_score = score
            best_k = k
print(best_p)
print(best_score)
print(best_k)

2
0.9888888888888889
3
Wall time: 23.7 s


# 网格搜索 Grib Search

In [38]:
param_grid = [
    {
        'weights': ['uniform'],
        'n_neighbors': [i for i in range(1, 11)]
    },
    {
        'weights': ['distance'],
        'n_neighbors': [i for i in range(1, 11)],
        'p': [i for i in range(1, 6)]
    }
]

In [29]:
knn_clf = KNeighborsClassifier()

In [30]:
from sklearn.model_selection import GridSearchCV
# CV 交叉验证
grid_search = GridSearchCV(knn_clf, param_grid)

In [32]:
%%time
grid_search.fit(X_train, y_train)

c:\program files\python37\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Wall time: 1min 14s


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform']},
                         {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [33]:
grid_search.best_estimator_ # 不是由用户传入的参数, 而是由类计算出来的结果, 用_xxx_命名

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=3,
                     weights='distance')

In [34]:
grid_search.best_score_ # 最佳结果

0.9853862212943633

In [35]:
grid_search.best_params_

{'n_neighbors': 3, 'p': 3, 'weights': 'distance'}

In [36]:
knn_clf = grid_search.best_estimator_
knn_clf.score(X_test, y_test)

0.9833333333333333

In [44]:
%%time
grid_search = GridSearchCV(knn_clf, param_grid, n_jobs=-1, verbose=2)
# n_jobs表示用几个核计算, -1表示将所有的核用于计算
# verbose运算过程中输出搜索结果, 数越大输出越多
grid_search.fit(X_train, y_train)

c:\program files\python37\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   18.5s


Wall time: 22.6 s


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   22.2s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=3, p=3,
                                            weights='distance'),
             iid='warn', n_jobs=-1,
             param_grid=[{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'weights': ['uniform']},
                         {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)